In [30]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [31]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [32]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [37]:
df_books[df_books.title == "Where the Heart Is (Oprah's Book Club (Paperback))"]

,isbn,title,author
706,0446672211,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts


In [48]:
# Clean the data
df_books.columns = ['ISBN', 'bookTitle', 'bookAuthor']
df_ratings.columns = ['userID', 'ISBN', 'bookRating']

# Merge the dataframes
data = pd.merge(df_ratings, df_books, on='ISBN')

# Perform data filtering to remove books and users with low ratings
book_ratings_counts = data['bookTitle'].value_counts()
user_ratings_counts = data['userID'].value_counts()

data = data[data['bookTitle'].isin(book_ratings_counts[book_ratings_counts >= 100].index)]
data = data[data['userID'].isin(user_ratings_counts[user_ratings_counts >= 200].index)]

In [49]:
data

,userID,ISBN,bookRating,bookTitle,bookAuthor
63,278418,0446520802,0.0,The Notebook,Nicholas Sparks
65,3363,0446520802,0.0,The Notebook,Nicholas Sparks
66,7158,0446520802,10.0,The Notebook,Nicholas Sparks
69,11676,0446520802,10.0,The Notebook,Nicholas Sparks
74,23768,0446520802,6.0,The Notebook,Nicholas Sparks
...,...,...,...,...,...
1027962,269566,0670809381,0.0,Echoes,Maeve Binchy
1028816,271284,0440910927,0.0,The Rainmaker,John Grisham
1029109,271705,B0001PIOX4,0.0,Fahrenheit 451,Ray Bradbury
1030863,275970,0865714215,0.0,Stormy Weather,Guy Dauncey


In [59]:
# Pivot the dataframe to create a user-book rating matrix
user_book_matrix = pd.pivot_table(data, index='bookTitle', columns='userID', values='bookRating', aggfunc='mean').fillna(0)
user_book_matrix

userID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
bookTitle,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# Train the K-Nearest Neighbors model:
knn = NearestNeighbors(n_neighbors=4, metric='cosine')
knn.fit(user_book_matrix)

NearestNeighbors(metric='cosine', n_neighbors=4)

In [86]:
def get_recommends(book_title):
    # Find the index of the given book title
    book_index = user_book_matrix.index.get_loc(book_title)
    
    # Find the nearest neighbors and their distances
    distances, indices = knn.kneighbors(user_book_matrix.iloc[book_index, :].values.reshape(1, -1))
    
    # Create a list of recommended books and their distances
    recommended_books = []
    for i in range(1, len(distances.flatten())):
        recommended_books.append([user_book_matrix.index[indices.flatten()[i]], distances.flatten()[i]])
    
    # Return the list of recommended books
    return [book_title, recommended_books]

In [87]:
books = get_recommends("The Notebook")
print(books)

['The Notebook', [['A Walk to Remember', 0.6674524], ['The Rescue', 0.71349746], ['One Door Away from Heaven', 0.7449809]]]


In [88]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.7145698], ['The Lovely Bones: A Novel', 0.7158567], ['I Know This Much Is True', 0.7496688]]]
You haven't passed yet. Keep trying!
